In [3]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from pennylane.templates import StronglyEntanglingLayers
from functools import partial

class VQGenerator:
    def __init__(self, n_qubits, n_layers, scaling, device='lightning.gpu', data_reuploading=False):
        self.dev = qml.device(device, wires=n_qubits)
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.scaling = scaling
        self.data_reuploading = data_reuploading
        self.initialize_weights()

    def initialize_weights(self):
        """Initialize the weights for the variational circuit."""
        self.weights = np.random.uniform(low=0, high=2 * np.pi, size=(self.n_layers + 1, self.n_qubits, 3), requires_grad=True)
    
    def S(self, x):
        """Data-encoding circuit block."""
        for w in range(self.n_qubits):
            qml.RX(self.scaling * x, wires=w)

    def W(self, theta):
        """Trainable circuit block."""
        if self.n_qubits == 1:
            qml.Rot(theta[0][0], theta[0][1], theta[0][2], wires=0)
        else:
            StronglyEntanglingLayers(theta.reshape((-1, self.n_qubits, 3)), wires=range(self.n_qubits))

    def compute_single_output(self, weights, x):
        """Compute the quantum model output for a given input x."""
        if self.data_reuploading:
            @qml.qnode(self.dev)
            def circuit():
                for layer in range(self.n_layers):
                    self.W(weights[layer])
                    self.S(x)
                self.W(weights[-1])
                return qml.probs(wires=list(range(self.n_qubits)))
        else:
            @qml.qnode(self.dev)
            def circuit():
                self.S(x)
                for layer in range(self.n_layers):
                    self.W(weights[layer])
                self.W(weights[-1])
                return qml.probs(wires=list(range(self.n_qubits)))
        return circuit()
    
    def compute_batch_outputs(self, weights, x_batch):
        """Compute the quantum model outputs for a batch of inputs."""
        if self.data_reuploading:
            @qml.batch_params
            @qml.qnode(self.dev)
            def circuit(x_batch):
                for layer in range(self.n_layers):
                    self.W(weights[layer])
                    for x in x_batch:
                        self.S(x)
                self.W(weights[-1])
                return [qml.expval(qml.PauliZ(wires=0)) for _ in range(len(x_batch))]
        else:
            @qml.batch_params
            @qml.qnode(self.dev)
            def circuit(x_batch):
                for x in x_batch:
                    self.S(x)
                for layer in range(self.n_layers):
                    self.W(weights[layer])
                self.W(weights[-1])
                return [qml.expval(qml.PauliZ(wires=0)) for _ in range(len(x_batch))]
        
        return circuit(x_batch)

    def visualize_quantum_circuit(self, x):
        """Visualize the quantum circuit for a given input."""
        if self.data_reuploading:
            @qml.qnode(self.dev)
            def circuit():
                for layer in range(self.n_layers):
                    self.W(self.weights[layer])
                    self.S(x)
                self.W(self.weights[-1])
                return qml.probs(wires=list(range(self.n_qubits)))
        else:
            @qml.qnode(self.dev)
            def circuit():
                self.S(x)
                for layer in range(self.n_layers):
                    self.W(self.weights[layer])
                self.W(self.weights[-1])
                return qml.probs(wires=list(range(self.n_qubits)))
        
        drawer = qml.draw(circuit, show_all_wires=True)
        print(drawer())

class Trainer:
    def __init__(self, model: VQGenerator, valid_state_mask: np.array,max_steps=200, batch_size=32, opt="Adam", learning_rate=0.2):
        self.model = model
        self.valid_state_mask = valid_state_mask
        self.max_steps = max_steps
        self.batch_size = batch_size
        self.opt = self.get_optimizer(opt, learning_rate)

    def get_optimizer(self, opt, learning_rate):
        """Retrieve the specified optimizer."""
        if opt == "Adam":
            return qml.AdamOptimizer(learning_rate)
        elif opt == "GradientDescent":
            return qml.GradientDescentOptimizer(learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {opt}")

    def valid_state_loss(self, target_probability):
        """ Need to test."""
        return - np.log(np.sum(target_probability * self.valid_state_mask))

    def cost(self, weights, x, targets):
        """Calculate the cost for a given set of weights and data."""
        predictions = [self.model.compute_quantum_model(weights, x_) for x_ in x]
        return self.square_loss(targets, predictions)

    def train(self, x, target_y):
        """Train the model using the provided data."""
        cst_history = [self.cost(self.model.weights, x, target_y)]  # Initial cost
        for step in range(self.max_steps):
            batch_indices = np.random.randint(0, len(x), self.batch_size)
            x_batch = x[batch_indices]
            y_batch = target_y[batch_indices]
            
            self.model.weights, _, _ = self.opt.step(self.cost, self.model.weights, x_batch, y_batch)
            
            current_cost = self.cost(self.model.weights, x, target_y)
            cst_history.append(current_cost)
            if (step + 1) % 10 == 0:
                print(f"Cost at step {step + 1:3}: {current_cost:.4f}")
        
        return self.model, cst_history


In [4]:
qc_model = VQGenerator(n_qubits=12, n_layers=2, scaling=1)
qc_model.visualize_quantum_circuit(1)


DeviceError: Device lightning.tensor does not exist. Make sure the required plugin is installed.

In [44]:
import sys 
sys.path.append("../")
from qmg.utils import MoleculeQuantumStateGenerator
import pandas as pd

data_path = "../dataset/chemical_space/effective_3.csv"
data = pd.read_csv(data_path)
data.head()

qg = MoleculeQuantumStateGenerator(heavy_atom_size=3)
quantum_state = qg.decimal_to_binary(64, 12)
qg.ConnectivityToSmiles(*qg.QuantumStateToConnectivity(quantum_state))

'C'

In [45]:
valid_state_mask = np.zeros(2**12, requires_grad=False)
for index, row in data.iterrows():
    valid_state_mask[-1-int(row["decimal_index"])] = 1.
print(valid_state_mask.shape)
valid_state_mask

(4096,)


tensor([0., 0., 0., ..., 0., 0., 0.], requires_grad=False)

In [57]:
batch_size = 10
weights = np.random.uniform(low=0, high=2 * np.pi, size=(batch_size, 2 + 1, 12, 3), requires_grad=True)
batch_inputs = np.random.uniform(size=(batch_size, 12))
ouput_prob = qc_model.compute_batch_outputs(weights, batch_inputs)
print(ouput_prob.shape)

ValueError: Parameter 3.9299758947039023 has incorrect batch dimension. Expecting first dimension of length 12.

In [61]:
weights

tensor([[[[3.92997589, 1.35666068, 0.22954502],
          [0.08401884, 1.81974524, 1.08668119],
          [0.33192181, 1.3343856 , 2.90679703],
          ...,
          [1.62366172, 4.37264211, 3.028859  ],
          [1.21531161, 4.88296065, 5.5592064 ],
          [2.23690708, 4.07273631, 4.52616824]],

         [[1.60851421, 2.20274593, 0.62835107],
          [1.73809578, 0.13265767, 4.34247816],
          [6.09129257, 2.39500429, 4.77780613],
          ...,
          [4.54979842, 1.62818812, 5.58295418],
          [3.37028542, 5.58444903, 2.76768498],
          [5.15848425, 0.25098391, 5.26054856]],

         [[1.43240584, 2.54273178, 3.31172584],
          [3.47498011, 5.27946618, 3.73341164],
          [1.38563223, 1.43734209, 2.8437527 ],
          ...,
          [3.27760665, 2.61895511, 6.18387143],
          [0.01849497, 3.54553244, 0.24969099],
          [5.50216998, 2.1056025 , 2.50393841]]],


        [[[1.90803741, 1.60979243, 4.76475759],
          [3.61869927, 1.48492182, 

In [63]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from pennylane.templates import StronglyEntanglingLayers

class VQC:
    def __init__(self, n_qubits, n_layers, scaling):
        self.dev = qml.device('lightning.gpu', wires=n_qubits)
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.scaling = scaling
        self.initialize_weights()

    def initialize_weights(self):
        """Initialize the weights for the variational circuit."""
        self.weights = np.random.uniform(low=0, high=2 * np.pi, size=(self.n_layers + 1, self.n_qubits, 3), requires_grad=True)
    
    def S(self, x):
        """Data-encoding circuit block."""
        for w in range(self.n_qubits):
            qml.RX(self.scaling * x, wires=w)

    def W(self, theta):
        """Trainable circuit block."""
        if self.n_qubits == 1:
            qml.Rot(theta[0][0], theta[0][1], theta[0][2], wires=0)
        else:
            StronglyEntanglingLayers(theta.reshape((-1, self.n_qubits, 3)), wires=range(self.n_qubits))

    def quantum_model(self, weights, x):
        """Define the quantum model for a given input x."""
        @qml.qnode(self.dev)
        def circuit(x):
            for layer in range(self.n_layers):
                self.W(weights[layer])
                self.S(x)
            self.W(weights[-1])
            return qml.expval(qml.PauliZ(wires=0))
        return circuit(x)
    
    def compute_batch_outputs(self, weights, x_batch):
        """Compute the quantum model outputs for a batch of inputs."""
        @qml.batch_params
        @qml.qnode(self.dev)
        def circuit(x_batch):
            for layer in range(self.n_layers):
                self.W(weights[layer])
                for x in x_batch:
                    self.S(x)
            self.W(weights[-1])
            return [qml.expval(qml.PauliZ(wires=0)) for _ in range(len(x_batch))]
        
        return circuit(x_batch)

class Trainer:
    def __init__(self, model: VQC, max_steps=200, batch_size=32, opt="Adam", learning_rate=0.2):
        self.model = model
        self.max_steps = max_steps
        self.batch_size = batch_size
        self.opt = self.get_optimizer(opt, learning_rate)

    def get_optimizer(self, opt, learning_rate):
        """Retrieve the specified optimizer."""
        if opt == "Adam":
            return qml.AdamOptimizer(learning_rate)
        elif opt == "GradientDescent":
            return qml.GradientDescentOptimizer(learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {opt}")

    def square_loss(self, targets, predictions):
        """Calculate the square loss between targets and predictions."""
        return 0.5 * np.mean((targets - predictions) ** 2)

    def cost(self, weights, x_batch, y_batch):
        """Calculate the cost for a given set of weights and batch of data."""
        predictions = self.model.compute_batch_outputs(weights, x_batch)
        return self.square_loss(y_batch, predictions)

    def train(self, x, y):
        """Train the model using the provided data."""
        cst = [self.cost(self.model.weights, x, y)]  # Initial cost
        for step in range(self.max_steps):
            batch_indices = np.random.randint(0, len(x), self.batch_size)
            x_batch = x[batch_indices]
            y_batch = y[batch_indices]
            
            self.model.weights, _, _ = self.opt.step(self.cost, self.model.weights, x_batch, y_batch)
            
            current_cost = self.cost(self.model.weights, x, y)
            cst.append(current_cost)
            if (step + 1) % 10 == 0:
                print(f"Cost at step {step + 1:3}: {current_cost:.4f}")
        
        return self.model, cst

# Example usage

# Define the parameters for the model
n_qubits = 2
n_layers = 3
scaling = 1.0

# Create a VQC instance
vqc = VQC(n_qubits, n_layers, scaling)

# Generate some random input data
np.random.seed(42)
x_data = np.random.uniform(low=0, high=2 * np.pi, size=(100,))  # 100 random input data points
y_data = np.sin(x_data)  # Target data (for example purposes, here using a sine function)

# Initialize the Trainer
trainer = Trainer(model=vqc, max_steps=100, batch_size=10, opt="Adam", learning_rate=0.1)

# Train the model
trained_model, training_costs = trainer.train(x_data, y_data)

# Plot the training costs
plt.plot(training_costs)
plt.xlabel("Step")
plt.ylabel("Cost")
plt.title("Training Cost Over Time")
plt.show()


ValueError: Parameter 1.4039230771439868 does not contain a batch dimension.

In [38]:
def compute_single_output(self, weights, x):
    """Compute the quantum model output for a given input x."""
    @qml.qnode(self.dev)
    def circuit():
        self.S(x)
        for layer in range(self.n_layers):
            self.W(weights[layer])
        self.W(weights[-1])
        return qml.probs(wires=list(range(self.n_qubits)))
    return circuit()

(10, 12)